# Libraries needed

In [1]:
# Import basic libraries
import dask.distributed as dsk
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

# Plotting libraries
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import cmocean as cm
import matplotlib.pyplot as plt

# Start client
clnt = dsk.Client()
clnt

2022-08-11 16:22:34,197 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-wemoi5qy', purging
2022-08-11 16:22:34,198 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-u1lwsakq', purging
2022-08-11 16:22:34,198 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-82ch7hlh', purging
2022-08-11 16:22:34,199 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-ksacgpta', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 8,Total memory: 22.46 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39275,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 8
Started: Just now,Total memory: 22.46 GiB
Comm: tcp://127.0.0.1:46115,Total threads: 2
Dashboard: /proxy/39761/status,Memory: 5.62 GiB
Nanny: tcp://127.0.0.1:43159,


# Daily averaged data

In [2]:
df = pd.read_csv('/g/data/e14/jn8053/andy-pitman/islands_AUS_export.csv')
locations = df.to_xarray()
extracted_data = xr.Dataset(data_vars = {'ulm_ID':locations['ulm_ID'],'long':locations['long'],'lat':locations['lat']})

- Load the required Lat & Long coordinates
- Resample the data from hourly to daily

In [ ]:
def preprocessing(ds):
    # Preprocess to extract everything above 70S
    ds = ds.sel(time = slice('2005-01-01', '2020-01-01'), longitude = slice(-156, 156), latitude = slice(90, -70))
    # Resample to daily via averaging
    ds = ds.resample(time = '1D').mean('time', keepdims=True)
    return ds

In [3]:
path_era5_daily = '/g/data/rt52/era5/single-levels/reanalysis/'

- Open the datasets and apply the preprocessing function

In [4]:
print('Opening: ')
listv = np.sort(glob.glob(path_era5_daily+'mtpr'+'/20*/*.nc'))
#data[v] = xr.open_mfdataset(listv, preprocess = preprocessing, chunks = {'latitude': 10, 'longitude': 10})
ds = xr.open_mfdataset(listv, chunks = {'time': -1})

Opening: 


In [5]:
ds

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, time: 195744)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 2000-01-01 ... 2022-04-30T23:00:00
Data variables:
    mtpr       (time, latitude, longitude) float32 dask.array<chunksize=(744, 721, 1440), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2020-10-07 10:06:24 UTC+1100 by era5_replication_tools-1.2....
    license:      Licence to use Copernicus Products: https://apps.ecmwf.int/...
    summary:      ERA5 is the fifth generation ECMWF atmospheric reanalysis o...
    title:        ERA5 single-levels reanalysis mean_total_precipitation_rate...

In [6]:
ds = ds.sel(time = slice('2005-01-01', '2020-01-01'), longitude = slice(-156, 156), latitude = slice(90, -70))
ds

<xarray.Dataset>
Dimensions:    (longitude: 1249, latitude: 641, time: 131496)
Coordinates:
  * longitude  (longitude) float32 -156.0 -155.8 -155.5 ... 155.5 155.8 156.0
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -69.5 -69.75 -70.0
  * time       (time) datetime64[ns] 2005-01-01 ... 2020-01-01T23:00:00
Data variables:
    mtpr       (time, latitude, longitude) float32 dask.array<chunksize=(744, 641, 1249), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2020-10-07 10:06:24 UTC+1100 by era5_replication_tools-1.2....
    license:      Licence to use Copernicus Products: https://apps.ecmwf.int/...
    summary:      ERA5 is the fifth generation ECMWF atmospheric reanalysis o...
    title:        ERA5 single-levels reanalysis mean_total_precipitation_rate...

In [7]:
mtpr = ds.mtpr.resample(time = '1D').mean('time', keepdims=True)